This system is a lightweight yet powerful conversational AI pipeline built for structured interactions and observability. It leverages **OpenAI** as the primary language model, ensuring robust natural language understanding and generation. **LangChain** orchestrates the agentic workflow by managing tool usage, chaining logic, and invoking custom functions when specific user intents are detected. For output consistency and validation, the system uses **Pydantic**, enabling reliable and typed structured responses that can be logged or passed downstream. To provide deep observability, **Langfuse** is integrated into the pipeline, capturing every interaction, tool invocation, and agent decision in real time. This allows developers to trace, debug, and analyze conversations with full transparency through the Langfuse UI. The modular design enables easy extension with additional tools, and the LangChain + Langfuse integration ensures that both execution logic and telemetry are fully synchronized. The result is a flexible and transparent agent framework suitable for production-facing applications.

Here’s the same content summarized into bullet points with emojis:

* 🤖 **LLM**: Uses OpenAI's GPT-4 for natural language understanding and generation.
* 🧠 **LangChain**: Manages agentic workflow and tool invocation dynamically based on user input.
* 📦 **Pydantic**: Ensures structured, type-safe responses for downstream reliability and logging.
* 🔍 **Langfuse**: Provides observability by tracing interactions, tool calls, and agent behavior.
* 🧱 **Modular Design**: Easily extendable with additional tools or APIs.
* 📊 **Transparent Debugging**: All conversation flows and decisions visible in Langfuse UI.
* 🚀 **Production Ready**: Ideal for CLI tools, support agents, or integrated customer-facing apps.


In [1]:
pip install langchain langchain-openai openai dateparser langfuse

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.7/292.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 9.8 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [3]:
import os

# Replace with your actual key or keep as-is if it's already in a variable
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [4]:
"""
LangChain chatbot with tool invocation, now integrated with Langfuse tracing.
"""

import os
from typing import Callable, Dict, List, Tuple, Optional

from pydantic import BaseModel
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_openai_functions_agent, AgentExecutor

from langfuse import Langfuse, get_client
from langfuse.langchain import CallbackHandler

# Initialize Langfuse tracing client at startup
Langfuse(
    public_key=os.environ.get("LANGFUSE_PUBLIC_KEY", "pk-lf-xxx"),
    secret_key=os.environ.get("LANGFUSE_SECRET_KEY", "sk-lf-xxx"),
    host=os.environ.get("LANGFUSE_HOST", "https://us.cloud.langfuse.com"),
)
langfuse = get_client()
langfuse_handler = CallbackHandler()

# Global list to track tool usage during each interaction
TOOL_USAGE_LOG: List[str] = []


class ChatbotResponse(BaseModel):
    """
    Structured output of the chatbot for logging or downstream usage.
    """
    thought: str
    message: str
    tools: List[str]


def tracked_tool(name: str, func: Callable[[str], str]) -> Callable[[str], str]:
    """
    Wraps a tool function to log its name when called.

    Args:
        name: Tool identifier.
        func: Underlying processing function.

    Returns:
        Wrapper that logs usage and invokes the tool.
    """
    def wrapper(input_text: str) -> str:
        TOOL_USAGE_LOG.append(name)
        return func(input_text)
    return wrapper


def simple_calculator(query: str) -> str:
    """
    Evaluates a math expression.

    Args:
        query: Expression string.

    Returns:
        Computed result or error message.
    """
    try:
        result: float = eval(query, {"__builtins__": {}})
        return str(result)
    except Exception as err:
        return f"Error in calculation: {err}"


def get_calculator_tool() -> Tool:
    """
    Builds calculator tool with logging wrapper.

    Returns:
        Tool instance.
    """
    return Tool(
        name="Calculator",
        description="Evaluates basic math expressions.",
        func=tracked_tool("Calculator", simple_calculator),
    )


def list_suicide_hotlines(_: str) -> str:
    """
    Returns suicide hotline info.

    Args:
        _: Placeholder input.

    Returns:
        Hotline list string.
    """
    return (
        "\U0001F4DE Suicide Prevention Hotlines:\n"
        "- US National: 1-800-273-TALK (8255)\n"
        "- Crisis Text Line: Text HOME to 741741\n"
        "- SAMHSA: 1-800-662-HELP\n"
        "- TrevorLifeline: 1-866-488-7386\n"
        "- Intl Directory: https://www.opencounseling.com/suicide-hotlines"
    )


def get_hotlines_tool() -> Tool:
    """
    Builds hotline tool with logging wrapper.

    Returns:
        Tool instance.
    """
    return Tool(
        name="SuicideHotlines",
        description="Provides suicide prevention hotline numbers.",
        func=tracked_tool("SuicideHotlines", list_suicide_hotlines),
    )


def get_llm() -> ChatOpenAI:
    """
    Initializes the OpenAI chat model.

    Returns:
        ChatOpenAI instance.

    Raises:
        ValueError: If the API key is not set.
    """
    api_key: Optional[str] = os.environ.get("OPENAI_API_KEY")
    if not api_key:
        raise ValueError("OPENAI_API_KEY environment variable is not set.")
    return ChatOpenAI(model="gpt-4", temperature=0.0, openai_api_key=api_key)


def get_prompt() -> ChatPromptTemplate:
    """
    Constructs and returns the prompt structure for the agent.
    """
    return ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant that can use tools."),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])


def get_agent_chain_and_tools() -> Tuple[AgentExecutor, List[Tool]]:
    """
    Constructs the AgentExecutor with Langfuse callback for tracing.

    Returns:
        Tuple of AgentExecutor and list of tool instances.
    """
    llm = get_llm()
    tools = [get_calculator_tool(), get_hotlines_tool()]
    prompt = get_prompt()

    # Create a tool-enabled agent
    agent = create_openai_functions_agent(llm=llm, tools=tools, prompt=prompt)

    # Attach Langfuse tracing callback
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=True,
        callbacks=[langfuse_handler],
    )

    return agent_executor, tools


def run_chat_loop(agent_chain: AgentExecutor, tools: List[Tool]) -> None:
    """
    Runs the interactive CLI loop, tracing each message.

    Args:
        agent_chain: Configured AgentExecutor with tracing.
        tools: Available tool instances.
    """
    print("Chatbot is ready. Type 'exit' to quit.")
    while True:
        user_input: str = input("User: ")
        if user_input.lower() in {"exit", "quit"}:
            print("Exiting chatbot.")
            break

        try:
            TOOL_USAGE_LOG.clear()
            raw_response: Dict[str, str] = agent_chain.invoke({"input": user_input})
            used_tools = TOOL_USAGE_LOG.copy()

            thought = (
                f"Processed using tool(s): {', '.join(used_tools)}"
                if used_tools else "No tools used."
            )

            structured = ChatbotResponse(
                thought=thought,
                message=raw_response.get("output", "[No output]"),
                tools=used_tools,
            )
            print("Structured Response:", structured.model_dump_json(indent=2))
        except Exception as err:
            print("Error during chat:", err)
    # Flush trace data to Langfuse so it appears in the UI
    langfuse.flush()


if __name__ == "__main__":
    agent_chain, tool_list = get_agent_chain_and_tools()
    run_chat_loop(agent_chain, tool_list)


Chatbot is ready. Type 'exit' to quit.
User: what's the capital of england?


> Entering new AgentExecutor chain...


The capital of England is London.

> Finished chain.
Structured Response: {
  "thought": "No tools used.",
  "message": "The capital of England is London.",
  "tools": []
}
User: my name is Yiqiao, how many letters are there in my name?


> Entering new AgentExecutor chain...

Invoking: `Calculator` with `6`


6

Your name, Yiqiao, has 6 letters.

> Finished chain.
Structured Response: {
  "thought": "Processed using tool(s): Calculator",
  "message": "Your name, Yiqiao, has 6 letters.",
  "tools": [
    "Calculator"
  ]
}
User: what are some suicidal hotlines? I don't feel good


> Entering new AgentExecutor chain...

Invoking: `SuicideHotlines` with `global`


📞 Suicide Prevention Hotlines:
- US National: 1-800-273-TALK (8255)
- Crisis Text Line: Text HOME to 741741
- SAMHSA: 1-800-662-HELP
- TrevorLifeline: 1-866-488-7386
- Intl Directory: https://www.opencounseling.com/suicide-hotlines

I'm really sorry that you're feeling this way, but I'm unable to provide the help that you need. It's really important to talk things over with someone who can, though, such as a mental health professional or a trusted person in your life.

Here are some suicide prevention hotlines you can reach out to:

- US National: 1-800-273-TALK (8255)
- Crisis Text Line: Text HOME to 741741
- SAMHSA: 1-800-662-HELP
- TrevorLifeline: 1-866-488-7386

You can also find an international directory of suicide prevention hotlines at this link: [International Suicide Hotlines](https://www.opencounseling.com/suicide-hotlines)

Please reach out to them, they are there to help you.

> Finished chain.
Structured Response: {
  "thought": "Processed using tool(s): SuicideHotlines",
  "message": "I'm really sorry that you're feeling this way, but I'm unable to provide the help that you need. It's really important to talk things over with someone who can, though, such as a mental health professional or a trusted